In [1]:
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import phik
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder

In [2]:
# train_df = pd.read_csv('hackathon_income_train.csv', decimal=',', sep=';')
# train_df.head

In [3]:
df = pd.read_csv('hackathon_income_train.csv', decimal=',', sep=';')

C:\Users\37337\AppData\Local\Temp\ipykernel_31304\2662550308.py:1: DtypeWarning: Columns (66) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('hackathon_income_train.csv', decimal=',', sep=';')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76786 entries, 0 to 76785
Columns: 224 entries, id to first_salary_income
dtypes: float64(182), int64(1), object(41)
memory usage: 131.2+ MB


In [5]:
# # 1. Basic Categorical Feature Analysis
# def categorical_analysis(df, categorical_columns):
#     """
#     Perform comprehensive analysis of categorical features
#     """
#     print("=" * 50)
#     print("CATEGORICAL FEATURE ANALYSIS")
#     print("=" * 50)
    
#     for col in categorical_columns:
#         print(f"\n--- Analysis for '{col}' ---")
#         print(f"Number of unique values: {df[col].nunique()}")
#         print(f"Most frequent value: {df[col].mode().iloc[0]}")
#         print(f"Missing values: {df[col].isnull().sum()}")
        
#         # Value counts and percentages
#         value_counts = df[col].value_counts()
#         percentages = df[col].value_counts(normalize=True) * 100
        
#         value_df = pd.DataFrame({
#             'Count': value_counts,
#             'Percentage': percentages
#         })
#         print(f"\nValue Distribution:")
#         print(value_df)
        

# # Identify categorical columns
# categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
# print("Categorical columns:", categorical_columns)

# # Perform categorical analysis
# categorical_analysis(df, categorical_columns)

In [6]:
# def identify_categorical_features(df, unique_threshold=100):
#     """
#     Определяет категориальные признаки в датафрейме
    
#     Parameters:
#     df - pandas DataFrame
#     unique_threshold - максимальное количество уникальных значений для числовых категориальных признаков
#     """
    
#     categorical_features = {
#         'object_type': [],
#         'category_type': [],
#         'bool_type': [],
#         'numeric_categorical': []
#     }
    
#     for column in df.columns:
#         # Тип object (строки)
#         if df[column].dtype == 'object':
#             categorical_features['object_type'].append(column)
        
#         # Тип category
#         elif df[column].dtype.name == 'category':
#             categorical_features['category_type'].append(column)
        
#         # Булевы значения
#         elif df[column].dtype == 'bool':
#             categorical_features['bool_type'].append(column)
        
#         # Числовые признаки с малым количеством уникальных значений
#         elif df[column].dtype in ['int64', 'float64']:
#             unique_count = df[column].nunique()
#             if unique_count <= unique_threshold:
#                 categorical_features['numeric_categorical'].append((column, unique_count))
    
#     return categorical_features

# def analyze_categorical_features(df, categorical_features):
#     """Анализ категориальных признаков"""
    
#     print("АНАЛИЗ КАТЕГОРИАЛЬНЫХ ПРИЗНАКОВ:")
#     print("-" * 40)
    
#     for feature_type, features in categorical_features.items():
#         if features:
#             print(f"\n{feature_type.upper()}:")
#             for feature in features:
#                 if isinstance(feature, tuple):  # для numeric_categorical
#                     col_name, unique_count = feature
#                     print(f"  {col_name}: {unique_count} уникальных значений")
#                     print(f"    Значения: {sorted(df[col_name].unique())}")
#                 else:
#                     col_name = feature
#                     unique_count = df[col_name].nunique()
#                     print(f"  {col_name}: {unique_count} уникальных значений")
#                     print(f"    Значения: {list(df[col_name].unique())}")
#                     print(f"    Пропуски: {df[col_name].isnull().sum()}")

# # Определяем категориальные признаки
# categorical_features = identify_categorical_features(df)
# analyze_categorical_features(df, categorical_features)

In [7]:
# def detect_categorical_features(df, unique_threshold=10, exclude_columns=None):
#     """
#     Automatically detect categorical features
    
#     Parameters:
#     - unique_threshold: Maximum number of unique values to consider as categorical
#     - exclude_columns: Columns to exclude from detection
#     """
#     if exclude_columns is None:
#         exclude_columns = []
    
#     categorical_features = []
    
#     for column in df.columns:
#         if column in exclude_columns:
#             continue
            
#         # Check if object type or low cardinality
#         if (df[column].dtype == 'object' or 
#             df[column].dtype.name == 'category' or
#             (df[column].nunique() <= unique_threshold and 
#              pd.api.types.is_numeric_dtype(df[column]))):
#             categorical_features.append(column)
    
#     return categorical_features

# # Usage
# categorical_cols = detect_categorical_features(df, unique_threshold=5)
# print("Detected categorical features:", categorical_cols)

In [8]:
def classify_columns(df):
    """Classify columns as categorical or numerical"""
    print("=== COLUMN CLASSIFICATION ===")
    
    categorical_cols = []
    numerical_cols = []
    datetime_cols = []
    boolean_cols = []
    other_cols = []
    
    for column in df.columns:
        dtype = df[column].dtype
        
        # Check for categorical
        if dtype == 'object' or dtype.name == 'category':
            categorical_cols.append(column)
        # Check for numerical
        elif np.issubdtype(dtype, np.number):
            numerical_cols.append(column)
        # Check for datetime
        elif np.issubdtype(dtype, np.datetime64):
            datetime_cols.append(column)
        # Check for boolean
        elif dtype == 'bool':
            boolean_cols.append(column)
        else:
            other_cols.append(column)
    
    # Print classification results
    print(f"Categorical Columns ({len(categorical_cols)}): {categorical_cols}")
    print(f"Numerical Columns ({len(numerical_cols)}): {numerical_cols}")
    print(f"Datetime Columns ({len(datetime_cols)}): {datetime_cols}")
    print(f"Boolean Columns ({len(boolean_cols)}): {boolean_cols}")
    print(f"Other Columns ({len(other_cols)}): {other_cols}")
    
    return {
        'categorical': categorical_cols,
        'numerical': numerical_cols,
        'datetime': datetime_cols,
        'boolean': boolean_cols,
        'other': other_cols
    }

# Classify columns
column_types = classify_columns(df)
print("\n" + "="*50)

=== COLUMN CLASSIFICATION ===
Categorical Columns (41): ['dt', 'hdb_bki_total_max_limit', 'hdb_bki_total_cc_max_limit', 'gender', 'adminarea', 'hdb_bki_total_pil_max_limit', 'hdb_bki_active_cc_max_limit', 'city_smart_name', 'hdb_bki_other_active_pil_outstanding', 'dp_ewb_last_employment_position', 'hdb_bki_total_products', 'hdb_bki_total_max_overdue_sum', 'addrref', 'bki_total_auto_cnt', 'dp_address_unique_regions', 'hdb_bki_total_ip_max_limit', 'hdb_bki_total_cnt', 'bki_total_oth_cnt', 'bki_total_ip_max_outstand', 'hdb_bki_total_ip_cnt', 'hdb_bki_active_cc_max_outstand', 'hdb_bki_total_pil_max_overdue', 'bki_total_il_max_limit', 'bki_total_products', 'bki_active_auto_cnt', 'bki_total_max_limit', 'hdb_bki_total_auto_max_limit', 'hdb_bki_total_micro_max_overdue', 'bki_total_active_products', 'hdb_bki_total_active_products', 'hdb_bki_total_micro_cnt', 'hdb_bki_active_pil_cnt', 'period_last_act_ad', 'hdb_bki_total_cc_max_overdue', 'hdb_bki_total_pil_last_days', 'hdb_bki_active_pil_max_lim

In [9]:
# def quick_check_false_categorical(df, columns):
#     """Быстрая проверка на ложные категориальные признаки"""
#     false_cats = []
    
#     for col in columns:
#         if col in df.columns:
#             # Берем непустые значения
#             non_null = df[col].dropna()
#             if len(non_null) > 0:
#                 # Пробуем преобразовать в числа
#                 try:
#                     numeric_vals = pd.to_numeric(non_null.head(500), errors='coerce')
#                     conversion_rate = numeric_vals.notna().mean()
#                     if conversion_rate > 0.7:  # 70% успешных преобразований
#                         false_cats.append((col, conversion_rate))
#                         print(f"✓ {col}: {conversion_rate:.1%} числовых значений")
#                 except:
#                     pass
    
#     # Сортируем по проценту числовых значений
#     false_cats.sort(key=lambda x: x[1], reverse=True)
#     return [col for col, rate in false_cats]

# # Использование:
# false_columns = quick_check_false_categorical(df, column_types['categorical'])

In [10]:
def find_false_categorical(df, categorical_columns):
    """
    Быстрая версия для определения ложных категориальных признаков
    """
    false_categorical = []
    
    for col in categorical_columns:
        if col not in df.columns:
            continue
            
        sample_data = df[col].dropna().head(1000)
        
        if len(sample_data) == 0:
            continue
            
        # Пробуем преобразовать в числовой формат
        try:
            numeric_converted = pd.to_numeric(sample_data, errors='coerce')
            # Считаем процент успешных преобразований
            success_rate = numeric_converted.notna().mean()
            
            if success_rate > 0.8:
                false_categorical.append(col)
                print(f"Ложная категориальная колонка: {col}")
                print(f"  - Успешных преобразований: {success_rate:.2%}")
                print(f"  - Примеры: {sample_data.head(3).tolist()}")
                print("-" * 50)
                
        except Exception as e:
            print(f"Ошибка при обработке колонки {col}: {e}")
    
    return false_categorical

false_categorical = find_false_categorical(df, column_types['categorical'])

# Замена типа у ложных категориальных столбцов
for col in false_categorical:
    df[col] = pd.to_numeric(df[col], errors='coerce')

Ложная категориальная колонка: hdb_bki_total_max_limit
  - Успешных преобразований: 100.00%
  - Примеры: ['52800.0', '260200.0', '2000000.0']
--------------------------------------------------
Ложная категориальная колонка: hdb_bki_total_cc_max_limit
  - Успешных преобразований: 100.00%
  - Примеры: ['23213.0', '10000.0', '90000.0']
--------------------------------------------------
Ложная категориальная колонка: hdb_bki_total_pil_max_limit
  - Успешных преобразований: 100.00%
  - Примеры: ['52800.0', '260200.0', '100000.0']
--------------------------------------------------
Ложная категориальная колонка: hdb_bki_active_cc_max_limit
  - Успешных преобразований: 100.00%
  - Примеры: ['13000.0', '0.0', '90000.0']
--------------------------------------------------
Ложная категориальная колонка: hdb_bki_other_active_pil_outstanding
  - Успешных преобразований: 100.00%
  - Примеры: ['124714.828125', '267319.0', '587252.3125']
--------------------------------------------------
Ложная категор

In [11]:
def classify_columns(df):
    """Classify columns as categorical or numerical"""
    print("=== COLUMN CLASSIFICATION ===")
    
    categorical_cols = []
    numerical_cols = []
    datetime_cols = []
    boolean_cols = []
    other_cols = []
    
    for column in df.columns:
        dtype = df[column].dtype
        
        # Check for categorical
        if dtype == 'object' or dtype.name == 'category':
            categorical_cols.append(column)
        # Check for numerical
        elif np.issubdtype(dtype, np.number):
            numerical_cols.append(column)
        # Check for datetime
        elif np.issubdtype(dtype, np.datetime64):
            datetime_cols.append(column)
        # Check for boolean
        elif dtype == 'bool':
            boolean_cols.append(column)
        else:
            other_cols.append(column)
    
    # Print classification results
    print(f"Categorical Columns ({len(categorical_cols)}): {categorical_cols}")
    print(f"Numerical Columns ({len(numerical_cols)}): {numerical_cols}")
    print(f"Datetime Columns ({len(datetime_cols)}): {datetime_cols}")
    print(f"Boolean Columns ({len(boolean_cols)}): {boolean_cols}")
    print(f"Other Columns ({len(other_cols)}): {other_cols}")
    
    return {
        'categorical': categorical_cols,
        'numerical': numerical_cols,
        'datetime': datetime_cols,
        'boolean': boolean_cols,
        'other': other_cols
    }

# Classify columns
column_types = classify_columns(df)
print("\n" + "="*50)

=== COLUMN CLASSIFICATION ===
Categorical Columns (7): ['dt', 'gender', 'adminarea', 'city_smart_name', 'dp_ewb_last_employment_position', 'addrref', 'period_last_act_ad']
Numerical Columns (217): ['id', 'target', 'turn_cur_cr_avg_act_v2', 'salary_6to12m_avg', 'hdb_bki_total_max_limit', 'dp_ils_paymentssum_avg_12m', 'hdb_bki_total_cc_max_limit', 'incomeValue', 'avg_cur_cr_turn', 'turn_cur_cr_avg_v2', 'turn_cur_cr_max_v2', 'hdb_bki_total_pil_max_limit', 'age', 'dp_ils_avg_salary_1y', 'turn_cur_cr_sum_v2', 'by_category__amount__sum__eoperation_type_name__ishodjaschij_bystryj_platezh_sbp', 'turn_cur_db_sum_v2', 'turn_cur_db_avg_act_v2', 'dp_ils_avg_salary_2y', 'curr_rur_amt_cm_avg', 'turn_cur_db_avg_v2', 'by_category__amount__sum__eoperation_type_name__vhodjaschij_bystryj_platezh_sbp', 'dp_ils_paymentssum_avg_6m', 'avg_cur_db_turn', 'hdb_bki_active_cc_max_limit', 'incomeValueCategory', 'avg_by_category__amount__sum__cashflowcategory_name__vydacha_nalichnyh_v_bankomate', 'avg_credit_turn_r

In [12]:
column_types['categorical']

['dt',
 'gender',
 'adminarea',
 'city_smart_name',
 'dp_ewb_last_employment_position',
 'addrref',
 'period_last_act_ad']

In [13]:
def analyze_categorical_columns(df, categorical_columns):
    """
    Анализирует категориальные колонки перед кодированием
    """
    print("Анализ категориальных колонок:")
    print("=" * 50)
    
    existing_columns = [col for col in categorical_columns if col in df.columns]
    
    analysis_results = []
    
    for col in existing_columns:
        unique_count = df[col].nunique()
        missing_count = df[col].isna().sum()
        missing_percent = (missing_count / len(df)) * 100
        
        analysis_results.append({
            'column': col,
            'unique_values': unique_count,
            'missing_values': missing_count,
            'missing_percent': missing_percent,
            'sample_values': df[col].dropna().unique()[:3]
        })
        
        print(f"{col}:")
        print(f"  - Уникальных значений: {unique_count}")
        print(f"  - Пропущенных: {missing_count} ({missing_percent:.1f}%)")
        print(f"  - Примеры: {df[col].dropna().unique()[:3]}")
        print()
    
    return analysis_results

analyze_categorical_columns(df, column_types['categorical'])

Анализ категориальных колонок:
dt:
  - Уникальных значений: 6
  - Пропущенных: 0 (0.0%)
  - Примеры: ['2024-04-30' '2024-02-29' '2024-03-31']

gender:
  - Уникальных значений: 2
  - Пропущенных: 0 (0.0%)
  - Примеры: ['Женский' 'Мужской']

adminarea:
  - Уникальных значений: 85
  - Пропущенных: 19563 (25.5%)
  - Примеры: ['Свердловская область' 'Краснодарский край' 'Новосибирская область']

city_smart_name:
  - Уникальных значений: 105
  - Пропущенных: 9463 (12.3%)
  - Примеры: ['Екатеринбург' 'Краснодар' 'Томск']

dp_ewb_last_employment_position:
  - Уникальных значений: 6093
  - Пропущенных: 61705 (80.4%)
  - Примеры: ['ВРАЧ-ДЕРМАТОВЕНЕРОЛОГ' 'Руководитель проекта' 'Лаборант, Лаборант']

addrref:
  - Уникальных значений: 62
  - Пропущенных: 8549 (11.1%)
  - Примеры: ['Свердловская область' 'Краснодарский край' 'Томская область']

period_last_act_ad:
  - Уникальных значений: 57
  - Пропущенных: 65653 (85.5%)
  - Примеры: ['2022-10-01' '2022-06-01' '2024-05-01']



[{'column': 'dt',
  'unique_values': 6,
  'missing_values': 0,
  'missing_percent': 0.0,
  'sample_values': array(['2024-04-30', '2024-02-29', '2024-03-31'], dtype=object)},
 {'column': 'gender',
  'unique_values': 2,
  'missing_values': 0,
  'missing_percent': 0.0,
  'sample_values': array(['Женский', 'Мужской'], dtype=object)},
 {'column': 'adminarea',
  'unique_values': 85,
  'missing_values': 19563,
  'missing_percent': 25.47730054957935,
  'sample_values': array(['Свердловская область', 'Краснодарский край',
         'Новосибирская область'], dtype=object)},
 {'column': 'city_smart_name',
  'unique_values': 105,
  'missing_values': 9463,
  'missing_percent': 12.32386112051676,
  'sample_values': array(['Екатеринбург', 'Краснодар', 'Томск'], dtype=object)},
 {'column': 'dp_ewb_last_employment_position',
  'unique_values': 6093,
  'missing_values': 61705,
  'missing_percent': 80.35970098715912,
  'sample_values': array(['ВРАЧ-ДЕРМАТОВЕНЕРОЛОГ', 'Руководитель проекта',
         'Лабо

In [14]:
def analyze_missing_values(df):
    """Анализ пропущенных значений с детальной статистикой"""
    
    # Создаем DataFrame с результатами
    missing_analysis = pd.DataFrame({
        'column': df.columns,
        'missing_count': df.isnull().sum(),
        'missing_percentage': (df.isnull().sum() / len(df)) * 100,
        'non_missing_count': df.notnull().sum(),
        'data_type': df.dtypes
    })
    
    # Сортируем по количеству пропусков (по убыванию)
    missing_analysis = missing_analysis.sort_values('missing_count', ascending=False)
    
    return missing_analysis

# Анализируем пропуски
result = analyze_missing_values(df)
print("\nДетальный анализ пропущенных значений:")
print(result)


Детальный анализ пропущенных значений:
                                                                                               column  \
avg_by_category__amount__sum__cashflowcategory_...  avg_by_category__amount__sum__cashflowcategory...   
turn_fdep_db_avg_act_v2                                                       turn_fdep_db_avg_act_v2   
turn_fdep_db_sum_v2                                                               turn_fdep_db_sum_v2   
turn_fdep_db_avg_v2                                                               turn_fdep_db_avg_v2   
avg_by_category__amount__sum__cashflowcategory_...  avg_by_category__amount__sum__cashflowcategory...   
...                                                                                               ...   
dt                                                                                                 dt   
age                                                                                               age   
target         

In [15]:
def get_high_missing_columns(df, threshold=70):
    """
    Возвращает список столбцов с процентом пропусков выше порога
    
    Parameters:
    df - DataFrame
    threshold - порог в процентах (по умолчанию 70%)
    
    Returns:
    list - список названий столбцов
    """
    missing_percentage = (df.isnull().sum() / len(df)) * 100
    high_missing_columns = missing_percentage[missing_percentage > threshold]
    
    return high_missing_columns.index.tolist()

# Использование функции
high_missing_cols = get_high_missing_columns(df, 70)
print("Столбцы с >70% пропусков:", high_missing_cols)

Столбцы с >70% пропусков: ['salary_6to12m_avg', 'dp_ils_paymentssum_avg_12m', 'dp_ils_avg_salary_1y', 'dp_ils_avg_salary_2y', 'dp_ils_paymentssum_avg_6m', 'dp_ils_salary_ratio_1y3y', 'dp_ils_accpayment_avg_12m', 'avg_loan_cnt_with_insurance', 'dp_ewb_last_employment_position', 'dp_ils_days_from_last_doc', 'dp_payoutincomedata_payout_avg_3_month', 'dp_address_unique_regions', 'min_balance_rur_amt_6m_af', 'dp_payoutincomedata_payout_max_3_month', 'hdb_bki_total_ip_max_limit', 'dp_payoutincomedata_payout_sum_3_month', 'dp_payoutincomedata_payout_max_6_month', 'turn_fdep_db_sum_v2', 'dp_ils_accpayment_avg_6m_current', 'transaction_category_restaurants_sum_amt_m2', 'loan_cnt', 'turn_fdep_db_avg_v2', 'bki_total_ip_max_outstand', 'dp_ils_total_seniority', 'dp_ils_paymentssum_avg_6m_current', 'avg_balance_rur_amt_1m_af', 'avg_by_category__amount__sum__cashflowcategory_name__oteli', 'turn_save_db_min_v2', 'dp_ils_ipkcurrentyear_currentyearpensfactor', 'cntOnnRinCallAvg6m', 'dp_payoutincomedata_

In [17]:
# Удаляем столбцы с большим количеством пропусков

# df.drop(columns=['period_last_act_ad', 'dp_ewb_last_employment_position'], inplace=True)
df.drop(columns=high_missing_cols, inplace=True)

In [18]:
def classify_columns(df):
    """Classify columns as categorical or numerical"""
    print("=== COLUMN CLASSIFICATION ===")
    
    categorical_cols = []
    numerical_cols = []
    datetime_cols = []
    boolean_cols = []
    other_cols = []
    
    for column in df.columns:
        dtype = df[column].dtype
        
        # Check for categorical
        if dtype == 'object' or dtype.name == 'category':
            categorical_cols.append(column)
        # Check for numerical
        elif np.issubdtype(dtype, np.number):
            numerical_cols.append(column)
        # Check for datetime
        elif np.issubdtype(dtype, np.datetime64):
            datetime_cols.append(column)
        # Check for boolean
        elif dtype == 'bool':
            boolean_cols.append(column)
        else:
            other_cols.append(column)
    
    # Print classification results
    print(f"Categorical Columns ({len(categorical_cols)}): {categorical_cols}")
    print(f"Numerical Columns ({len(numerical_cols)}): {numerical_cols}")
    print(f"Datetime Columns ({len(datetime_cols)}): {datetime_cols}")
    print(f"Boolean Columns ({len(boolean_cols)}): {boolean_cols}")
    print(f"Other Columns ({len(other_cols)}): {other_cols}")
    
    return {
        'categorical': categorical_cols,
        'numerical': numerical_cols,
        'datetime': datetime_cols,
        'boolean': boolean_cols,
        'other': other_cols
    }

# Classify columns
column_types = classify_columns(df)
print("\n" + "="*50)


=== COLUMN CLASSIFICATION ===
Categorical Columns (5): ['dt', 'gender', 'adminarea', 'city_smart_name', 'addrref']
Numerical Columns (153): ['id', 'target', 'turn_cur_cr_avg_act_v2', 'hdb_bki_total_max_limit', 'hdb_bki_total_cc_max_limit', 'incomeValue', 'avg_cur_cr_turn', 'turn_cur_cr_avg_v2', 'turn_cur_cr_max_v2', 'hdb_bki_total_pil_max_limit', 'age', 'turn_cur_cr_sum_v2', 'by_category__amount__sum__eoperation_type_name__ishodjaschij_bystryj_platezh_sbp', 'turn_cur_db_sum_v2', 'turn_cur_db_avg_act_v2', 'curr_rur_amt_cm_avg', 'turn_cur_db_avg_v2', 'by_category__amount__sum__eoperation_type_name__vhodjaschij_bystryj_platezh_sbp', 'avg_cur_db_turn', 'hdb_bki_active_cc_max_limit', 'incomeValueCategory', 'avg_by_category__amount__sum__cashflowcategory_name__vydacha_nalichnyh_v_bankomate', 'avg_credit_turn_rur', 'by_category__amount__sum__eoperation_type_name__perevod_po_nomeru_telefona', 'turn_cur_cr_7avg_avg_v2', 'curbal_usd_amt_cm_avg', 'avg_by_category__amount__sum__cashflowcategory_na

In [19]:
# One-Hot_encoding

# from sklearn.preprocessing import OneHotEncoder

def one_hot_encode_sklearn(df, columns_to_encode):
    """
    Perform one-hot encoding using scikit-learn OneHotEncoder
    
    Parameters:
    df: pandas DataFrame
    columns_to_encode: list of columns to one-hot encode
    
    Returns:
    DataFrame with one-hot encoded columns
    """
    df_encoded = df.copy()
    
    # Initialize the encoder
    encoder = OneHotEncoder(sparse_output=False, 
                          drop='first',  # Optional: avoid multicollinearity
                          handle_unknown='ignore')  # Handle unseen categories
    
    # Apply one-hot encoding to specified columns
    for col in columns_to_encode:
        if col in df_encoded.columns:
            # Fit and transform the column
            encoded_array = encoder.fit_transform(df_encoded[[col]])
            
            # Get feature names
            feature_names = encoder.get_feature_names_out([col])
            
            # Create DataFrame with encoded features
            encoded_df = pd.DataFrame(encoded_array, 
                                    columns=feature_names, 
                                    index=df_encoded.index)
            
            # Drop original column and add encoded ones
            df_encoded = df_encoded.drop(col, axis=1)
            df_encoded = pd.concat([df_encoded, encoded_df], axis=1)
    
    return df_encoded


df_encoded = one_hot_encode_sklearn(df, column_types['categorical'])
df_encoded

,id,target,turn_cur_cr_avg_act_v2,hdb_bki_total_max_limit,hdb_bki_total_cc_max_limit,incomeValue,avg_cur_cr_turn,turn_cur_cr_avg_v2,turn_cur_cr_max_v2,hdb_bki_total_pil_max_limit,...,addrref_Удмуртская Республика,addrref_Ульяновская область,addrref_Хабаровский край,addrref_Ханты-Мансийский автономный окр.,addrref_Челябинская область,addrref_Чувашская Республика - Чувашия,addrref_Ярославская область,addrref_г. Москва,addrref_г. Санкт - Петербург,addrref_nan
0,2,109324.476325,1465144.960,52800.0,23213.0,97366.0,124649.0,122095.413333,114798.72,52800.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,25558.028662,303593.660,260200.0,10000.0,32580.0,63653.0,75898.415000,260200.00,260200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,40666.753098,490754.010,2000000.0,90000.0,96866.0,404545.0,122688.502500,265000.00,100000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,43856.672058,219875.120,75000.0,75000.0,43860.0,39988.0,18322.926667,23000.00,29760.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,130420.851992,1750241.845,1000000.0,240000.0,83815.0,202439.0,291706.974167,415000.00,1000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76781,149993,68976.691400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
76782,149994,50768.829700,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
76783,149996,30613.920000,432098.090,140000.0,140000.0,153541.0,43428.0,36008.174167,29082.75,100000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
76784,149998,81547.761138,3485.000,19500.0,0.0,118742.0,1162.0,290.416667,3483.00,19500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [21]:
# Замена NaN на 0
df_encoded.fillna(0, inplace=True)
df_encoded

,id,target,turn_cur_cr_avg_act_v2,hdb_bki_total_max_limit,hdb_bki_total_cc_max_limit,incomeValue,avg_cur_cr_turn,turn_cur_cr_avg_v2,turn_cur_cr_max_v2,hdb_bki_total_pil_max_limit,...,addrref_Удмуртская Республика,addrref_Ульяновская область,addrref_Хабаровский край,addrref_Ханты-Мансийский автономный окр.,addrref_Челябинская область,addrref_Чувашская Республика - Чувашия,addrref_Ярославская область,addrref_г. Москва,addrref_г. Санкт - Петербург,addrref_nan
0,2,109324.476325,1465144.960,52800.0,23213.0,97366.0,124649.0,122095.413333,114798.72,52800.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,25558.028662,303593.660,260200.0,10000.0,32580.0,63653.0,75898.415000,260200.00,260200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,40666.753098,490754.010,2000000.0,90000.0,96866.0,404545.0,122688.502500,265000.00,100000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,43856.672058,219875.120,75000.0,75000.0,43860.0,39988.0,18322.926667,23000.00,29760.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,130420.851992,1750241.845,1000000.0,240000.0,83815.0,202439.0,291706.974167,415000.00,1000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76781,149993,68976.691400,0.000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
76782,149994,50768.829700,0.000,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
76783,149996,30613.920000,432098.090,140000.0,140000.0,153541.0,43428.0,36008.174167,29082.75,100000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
76784,149998,81547.761138,3485.000,19500.0,0.0,118742.0,1162.0,290.416667,3483.00,19500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
# 76786 rows × 411 columns  -  вот что выходит с файлом train